**Initial Configuration**

In [2]:
import sys
sys.path.append('..')

In [3]:
import nppes

**Begin Notebook Code**

# Get data from API

In [9]:
import requests

In [6]:
API_columns = [
    'number',
    'enumeration_type',
    'taxonomy_description',
    'first_name',
    'last_name',
    'organization_name',
    'address_purpose',
    'city',
    'state',
    'postal_code',
    'country_code',
    'limit',
    'skip',
    'pretty'
    ]

In [17]:
payload = {'version': '2.1', 'last_name': 'Gomez', 'first_name': 'Julia'}
r = requests.get('https://npiregistry.cms.hhs.gov/api/?', params=payload)

print(r.url)

https://npiregistry.cms.hhs.gov/api/?version=2.0&last_name=Gomez&first_name=Julia


In [18]:
r.content

b'{"result_count":3, "results":[{"enumeration_type": "NPI-1", "number": 1174576599, "last_updated_epoch": 1259712000, "created_epoch": 1147910400, "basic": {"name_prefix": "DR.", "first_name": "JULIA", "last_name": "GOMEZ", "middle_name": "E", "credential": "M.D.", "sole_proprietor": "NO", "gender": "F", "enumeration_date": "2006-05-18", "last_updated": "2009-12-02", "status": "A", "name": "GOMEZ JULIA"}, "other_names": [], "addresses": [{"country_code": "US", "country_name": "United States", "address_purpose": "LOCATION", "address_type": "DOM", "address_1": "2400 PATTERSON ST", "address_2": "500", "city": "NASHVILLE", "state": "TN", "postal_code": "372031562", "telephone_number": "615-327-7400", "fax_number": "615-327-4818"}, {"country_code": "US", "country_name": "United States", "address_purpose": "MAILING", "address_type": "DOM", "address_1": "2400 PATTERSON ST", "address_2": "500", "city": "NASHVILLE", "state": "TN", "postal_code": "372031562", "telephone_number": "615-327-7400", 

In [ ]:
def search_nppes():
    